In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
from utils.fehler_gauss import fehler_gauss, weighted_mean
from sympy import *

## Linsentypen

- A Sammellinse
- B Sammellinse
- G Sammellinse
- E Streulinse

## Allgemeine Settings

In [3]:
pos_lampe = 0e-2
u_pos_lampe = 0.1e-2

pos_linse_a = 5e-2
u_pos_linse_a = 0.1e-2

pos_gitter = 20e-2
u_pos_gitter = 0.1e-2

## Autokollimationsmethode Einfachlinsen

Brennweite: $f' = \frac{k + l -h}{2} \approx \frac{k + l}{2}$

In [4]:
d_linse_spiegel = 3e-2
u_d_linse_spiegel = 0.1e-2

autokoll_einfach = pd.read_excel("OPA.xlsx", sheet_name="AutokollimationEinfachlinsen") * 1e-2 # importieren in meter
u_autokoll_einfach = 0.3e-2 # ableseunsicherheit in meter

k = autokoll_einfach - pos_gitter # abstand gitter - bildschirm
l = k # abstand gitter - bildschirm umgedreht (nutzen einfach das Gleiche weil Gangsta)

In [5]:
# Typ A Unsicherheiten für die Abstände (korrigiert mit Student-t Verteilung)
u_k_typ_a = k.std(ddof=1) * 0.51
u_l_typ_a = l.std(ddof=1) * 0.51

# Gaussche Fehlerfortpflanzung für die Brennweite
k_symbol, l_symbol = symbols("k l")
brennweite_formula = (k_symbol + l_symbol) / 2
brennweite_formula

k/2 + l/2

In [6]:
gesamtunsicherheit_k = np.sqrt(u_k_typ_a**2 + u_autokoll_einfach**2)
gesamtunsicherheit_l = np.sqrt(u_l_typ_a**2 + u_autokoll_einfach**2)

brennweite, u_brennweite = fehler_gauss(
    brennweite_formula,
    [k_symbol, l_symbol],
    [k.mean(), l.mean()],
    [gesamtunsicherheit_k, gesamtunsicherheit_l],
)
brennweite * 1e2 # brennweite in cm

G     7.50
B    10.18
dtype: float64

In [7]:
u_brennweite * 1e2 # unsicherheit in cm

G    0.215176
B    0.214267
dtype: float64

## Besselmethode Einfachlinsen

$e = -a_1 + h + d + a_2' = -a_2 + h - d + a_1' = b_k - g_k$

$b_k$ Position Schirm beim kleinen Bild

$g$ Position Gitter

$d = l_k - l_g$ Abstand zwischen den rechten Hauptebenen von kleinem zu großem Bild

$l_k$ Position der Linse beim kleinen Bild

$l_g$ Position der Linse beim großen Bild

Brennweite $f' = \frac{1}{4} \cdot \left[(e - h) - \frac{d^2}{e - h} \right] \approx \frac{1}{4} \cdot \left[e - \frac{d^2}{e}\right]$ (dünne Linse)

In [26]:
gittergroesse = 5e-3
u_gittergroesse = 0.1e-2

gittergroesse_schirm = 20e-3
u_gittergroesse_schirm = 0.1e-3

g_pos_schirm = 69.5e-2
u_g_pos_schirm = 0.1e-2

e_pos_schirm = 84.1e-2
u_e_pos_schirm = 0.1e-2

bessel_einfach = pd.read_excel("OPA.xlsx", sheet_name="BesselEinfachlinsen") * 1e-2 # in meter
bessel_einfach

G gross  G klein  E gross  E klein
0    0.294    0.611    0.329    0.720
1    0.294    0.609    0.325    0.719
2    0.291    0.608    0.329    0.720
3    0.292    0.606    0.327    0.722
4    0.293    0.608    0.325    0.720

In [27]:
# gausssche Fehlerfortpflanzung für die Brennweite
b_k_symbol, g_symbol = symbols("b_k g")
e = b_k_symbol - g_symbol

l_k_symbol, l_g_symbol = symbols("l_k l_g")
d = l_k_symbol - l_g_symbol

brennweite_formula = 1/4 * (e - d**2/e)
brennweite_formula

0.25*b_k - 0.25*g - 0.25*(-l_g + l_k)**2/(b_k - g)

In [30]:
# Bessel für Linse G
brennweiten_g, u_brennweiten_g = fehler_gauss(
    brennweite_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol],
    [
        g_pos_schirm,
        pos_gitter,
        bessel_einfach["G klein"],
        bessel_einfach["G gross"],
    ],
    [u_g_pos_schirm, u_pos_gitter, 5e-3, 5e-3],
)
brennweite_g, u_brennweite_g = weighted_mean(brennweiten_g, u_brennweiten_g)
brennweite_g * 1e2, u_brennweite_g * 1e2 # Ergebnisse in cm

(np.float64(7.344739640914351), np.float64(0.10323126960090592))

In [32]:
# Bessel für Linse E
brennweiten_e, u_brennweiten_e = fehler_gauss(
    brennweite_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol],
    [
        e_pos_schirm,
        pos_gitter,
        bessel_einfach["E klein"],
        bessel_einfach["E gross"],
    ],
    [u_e_pos_schirm, u_pos_gitter, 5e-3, 5e-3],
)
brennweite_e, u_brennweite_e = weighted_mean(brennweiten_e, u_brennweiten_e)
brennweite_e * 1e2, u_brennweite_e * 1e2 # Ergebnisse in cm

(np.float64(9.995497858863873), np.float64(0.0993978008322837))

## Brennweite und Hauptebenenabstand Linsensystem mit Kombination aus Autokollimation und Bessel

$p_{12}$ Absolutposition von Linse G

### Autokollimation

$k = p_1 - g$

$l = p_2 - g$

### Besselmethode

$e = b_k - g$

$d = l_k - l_g$

### Insgesamt

$f' = \frac{1}{2} \sqrt{(e - k - l)^2 - d^2}$

$h = k + l - \sqrt{(e - k - l)^2 - d^2}$

In [42]:
autokoll_linsensystem = pd.read_excel("OPA.xlsx", sheet_name="AutokollimationLinsensystem") * 1e-2 # in meter
autokoll_linsensystem

G-E-Spiegel  E-G-Spiegel
0        0.366        0.308
1        0.365        0.306
2        0.367        0.308
3        0.366        0.305
4        0.367        0.306

In [54]:
g_e_pos_schirm = 107.5e-2
u_g_e_pos_schirm = 0.1e-2

e_g_pos_schirm = 104.6e-2
u_e_g_pos_schirm = 0.1e-2

bessel_linsensystem = pd.read_excel("OPA.xlsx", sheet_name="BesselLinsensystem") * 1e-2 # in meter
bessel_linsensystem

G-E-gross  G-E-klein  E-G-gross  E-G-klein
0      0.392      0.941      0.339      0.847
1      0.391      0.944      0.342      0.848
2      0.393      0.940      0.342      0.851
3      0.394      0.940      0.337      0.851
4      0.394      0.944      0.341      0.850

In [55]:
# fehler gauss
p1_symbol, p2_symbol, g_symbol = symbols("p_1 p_2 g")
k = p1_symbol - g_symbol
l = p2_symbol - g_symbol
brennweite_formula = 1/2 * sqrt((e - k - l)**2 - d**2)
h_formula = k + l - sqrt((e - k - l)**2 - d**2)
brennweite_formula

0.5*sqrt(-(-l_g + l_k)**2 + (b_k + g - p_1 - p_2)**2)

In [56]:
h_formula

-2*g + p_1 + p_2 - sqrt(-(-l_g + l_k)**2 + (b_k + g - p_1 - p_2)**2)

In [70]:
# Brennweite mit E G Schirm Konfiguration im Bessel
brennweite_system, u_brennweite_system = fehler_gauss(
    brennweite_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol, p1_symbol, p2_symbol],
    [e_g_pos_schirm, pos_gitter, bessel_linsensystem["E-G-klein"], bessel_linsensystem["E-G-gross"], autokoll_linsensystem["E-G-Spiegel"], autokoll_linsensystem["G-E-Spiegel"]],
    [u_e_g_pos_schirm, u_pos_gitter, 5e-3, 5e-3, 5e-3, 5e-3],
)

brennweite_system_e_g, u_brennweite_e_g = weighted_mean(brennweite_system, u_brennweite_system)
brennweite_system_e_g * 1e2, u_brennweite_e_g * 1e2 # Ergebnisse in cm

(np.float64(13.168635809189059), np.float64(0.46570172679655597))

In [71]:
# Brennweite mit G E Schirm Konfiguration im Bessel
brennweite_system, u_brennweite_system = fehler_gauss(
    brennweite_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol, p1_symbol, p2_symbol],
    [g_e_pos_schirm, pos_gitter, bessel_linsensystem["G-E-klein"], bessel_linsensystem["G-E-gross"], autokoll_linsensystem["E-G-Spiegel"], autokoll_linsensystem["G-E-Spiegel"]],
    [u_g_e_pos_schirm, u_pos_gitter, 5e-3, 5e-3, 5e-3, 5e-3],
)
brennweite_system_g_e, u_brennweite_g_e = weighted_mean(brennweite_system, u_brennweite_system)
brennweite_system_g_e * 1e2, u_brennweite_g_e * 1e2 # Ergebnisse in cm

(np.float64(12.383426871542818), np.float64(0.5262844151069097))

In [79]:
# Hauptebenenabstand mit E G Schirm Konfiguration im Bessel
h_system, u_h_system = fehler_gauss(
    h_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol, p1_symbol, p2_symbol],
    [e_g_pos_schirm, pos_gitter, bessel_linsensystem["E-G-klein"], bessel_linsensystem["E-G-gross"], autokoll_linsensystem["E-G-Spiegel"], autokoll_linsensystem["G-E-Spiegel"]],
    [u_e_g_pos_schirm, u_pos_gitter, 5e-3, 5e-3, 5e-3, 5e-3],
)

h_system_e_g, u_h_system_e_g = weighted_mean(h_system, u_h_system)
h_system_e_g * 1e2, u_h_system_e_g * 1e2 # Ergebnisse in cm

(np.float64(0.9472449307991704), np.float64(1.195374620144803))

In [80]:
# Hauptebenenabstand mit G E Schirm Konfiguration im Bessel
h_system, u_h_system = fehler_gauss(
    h_formula,
    [b_k_symbol, g_symbol, l_k_symbol, l_g_symbol, p1_symbol, p2_symbol],
    [e_g_pos_schirm, pos_gitter, bessel_linsensystem["G-E-klein"], bessel_linsensystem["G-E-gross"], autokoll_linsensystem["E-G-Spiegel"], autokoll_linsensystem["G-E-Spiegel"]],
    [u_e_g_pos_schirm, u_pos_gitter, 5e-3, 5e-3, 5e-3, 5e-3],
)

h_system_g_e, u_h_system_g_e = weighted_mean(h_system, u_h_system)
h_system_g_e * 1e2, u_h_system_g_e * 1e2 # Ergebnisse in cm

(np.float64(10.74696348827484), np.float64(1.7896018826720688))

## Abbeverfahren Linsensysteme

In [9]:
pos_schirm = 150e-2
u_pos_schirm = 0.1e-2

pd.read_excel("OPA.xlsx", sheet_name="AbbeE-G-Schirm")

Schirm  Vergroesserung  Position G Verkleinerung  Position G.1
0     150           34.40        33.0             ?         132.1
1     125           24.55        32.7             1         106.1
2     100           17.60        33.4          1.35          79.9
3      75            7.65        38.0           3.2          52.1

In [8]:
pd.read_excel("OPA.xlsx", sheet_name="AbbeG-E-Schirm")

Schirm  Vergroesserung  Position G Verkleinerung  Position G.1
0     150           33.75        39.5             ?         137.7
1     125           25.45        38.5           1.5         112.1
2     100           17.45        40.2           1.5          85.6
3      75            8.15        44.8           3.3          56.3